In [28]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 776.8 MB 26 bytes/s            
     |████████████████████████████████| 1.3 MB 115.4 MB/s            
     |████████████████████████████████| 62 kB 1.3 MB/s             
     |████████████████████████████████| 1.2 MB 90.2 MB/s            
     |████████████████████████████████| 2.9 MB 67.3 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.1+cu111 requires torch==1.9.1, but you have torch 1.7.0 which is incompatible.
torchaudio 0.9.1 requires torch==1.9.1, but you have torch 1.7.0 which 

In [115]:
import pandas as pd

In [116]:
dataset = pd.read_csv("dataset_512.tsv", sep="\t", header=None, names=["label", "text"])
dataset.head()

,label,text
0,Liberal,Thank you! [applause] Thank you! [applause] Th...
1,Liberal,and I thought I was doing pretty well until we...
2,Liberal,"More to the point, by the time I met her she h..."
3,Liberal,[applause]...she went to South Carolina to see...
4,Liberal,"And when she did, the people at the law school..."


In [126]:
data_sampled = pd.concat(objs=[dataset[dataset.label == 'Liberal'].sample(n=80000, random_state=42), 
                               dataset[dataset.label == 'Conservative'].sample(n=80000, random_state=42)])

In [ ]:
data_sampled.to_csv("dataset_512_train_160k.tsv", sep="\t", index=False, header=False)

In [127]:
#!g1.1
from run_pplm_discrim_train import train_discriminator

In [129]:
#!g1.1
discrim_args = {
    "dataset": "generic",
    "dataset_fp": "dataset_512_train_160k.tsv",
    "epochs": 5,
    "batch_size": 128,
    "learning_rate": 0.005,
    "log_interval": 128,
    "save_model": "True",
    "output_fp": "political_classificationhead_5_0005_160.pt"
}

discriminator, discriminator_meta = train_discriminator(**discrim_args)

160000it [00:05, 29676.22it/s]
160000it [04:47, 557.30it/s]


Preprocessing generic dataset...
Preprocessed 160000 data points
Data preprocessing took: 308.641s

Epoch 1
Train Epoch: 1 [128/144000 (0%)]	Loss: 3.271867
Train Epoch: 1 [16512/144000 (11%)]	Loss: 0.600420
Train Epoch: 1 [32896/144000 (23%)]	Loss: 0.502098
Train Epoch: 1 [49280/144000 (34%)]	Loss: 0.503613
Train Epoch: 1 [65664/144000 (46%)]	Loss: 0.516779
Train Epoch: 1 [82048/144000 (57%)]	Loss: 0.533724
Train Epoch: 1 [98432/144000 (68%)]	Loss: 0.522634
Train Epoch: 1 [114816/144000 (80%)]	Loss: 0.539873
Train Epoch: 1 [131200/144000 (91%)]	Loss: 0.507526
Performance on test set: Average loss: 0.5427, Accuracy: 11499/16000 (72%)
Epoch took: 5561.820s

Example prediction
Input sentence: Hate and fear are being given too much oxygen by those who pretend to love America, but who don’t understand America. To confront the ideology of hate requires caring about all people. That’s the America I know: the most diverse, multi-racial, dynamic nation in history.
Predictions: Conservative: 0.6

In [69]:
import torch
from pplm_classification_head import ClassificationHead

In [85]:
#!g1.1
model = ClassificationHead(2, 1024)
model.load_state_dict(torch.load("political_classificationhead_5_0005_160.pt/generic_classifier_head_epoch_5.pt"))
model.eval()

ClassificationHead(
  (mlp): Linear(in_features=1024, out_features=2, bias=True)
)